In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
  return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Modules
import scipy as sp
from scipy import sparse, signal, stats
from astropy import convolution as conv
import numpy as np
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
from matplotlib import colors as clrs
import pandas as pd 
import time
import datetime
import calendar
import sqlite3
from tqdm.autonotebook import tqdm

%matplotlib inline 



# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

# Font Size

font = {'family': 'sans-serif',
        'weight': 'normal',
        'size'   : 21}

plt.rc('font', **font)

C:\Users\Andres\AppData\Local\Temp\ipykernel_415504\1102906597.py:14: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Read file with largest interval

In [3]:
db = sqlite3.connect('data\SWAMIS\\100_frames\swamis_00045s_rotAssoc.sdb')
# db = sqlite3.connect('data\SWAMIS\swamisMDI.sdb')
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# tabs = pd.read_sql_query('SELECT * FROM tabs WHERE frame = 3173',db)
tabs = pd.read_sql_query('SELECT * FROM tabs',db)

[('params',), ('files',), ('tabs',), ('hist',), ('events',), ('event_hist',)]


In [4]:
list(tabs)

['id',
 'frame',
 'scale',
 'assoc_larger',
 'flux',
 'maxB',
 'str_pix',
 'size',
 'x',
 'y',
 'xerr',
 'yerr',
 'closeness',
 'est_dflux',
 'neighbors',
 'xs',
 'ys',
 'lon',
 'lat',
 'size_s']

### Clean elements that have a larger associated element

In [5]:
# Find elements that have an associated larger element
tabs_assLrg = tabs[np.isfinite(tabs['assoc_larger'])].copy().reset_index(drop=True)

# Find maks of all objects with an associated larger element in at least one frame
msk_assLrg = np.isin(tabs['id'],tabs_assLrg['id'])

# Create table with elements without an associated larger element
tabs_NoassLrg = tabs[np.logical_not(msk_assLrg)].copy().reset_index(drop=True)
tabs_NoassLrg

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,xerr,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s
0,1,1,1,NaN,-409.852418,-77.336472,0,2,2168.542486,3816.000000,0.498192,0.000043,0,-409.852418,None,-66.075702,-891.520053,-14.070494,-73.466620,0.508633
1,3,1,1,NaN,-401.259592,-74.707427,0,2,2042.486745,3810.000000,0.499824,0.000086,1,-401.259592,-1642,-2.502384,-888.572155,-0.515724,-73.077809,0.508633
2,4,1,1,NaN,-744.679039,-95.060790,1,3,1827.657345,3809.082584,0.335591,0.570318,1,-744.679039,-1648,105.836138,-888.242242,21.967109,-72.768028,0.762949
3,5,1,1,NaN,-412.025671,-77.163422,0,2,1877.470658,3809.470658,0.499138,0.499138,1,-412.025671,"-1647,11838",80.715163,-888.407168,16.705742,-72.902785,0.508633
4,6,1,1,NaN,-935.759582,-95.803365,1,4,1882.287636,3807.993468,0.261344,0.392628,1,-935.759582,"-1653,-1652,-1651,-1647,11838",78.286886,-887.659248,16.083357,-72.790183,1.017266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764471,553143,97,8,NaN,65.999160,32.961315,0,2,231.000000,247.502607,0.000000,0.499993,1,NaN,23056,-102.902421,-32.036954,-6.207114,-6.477249,32.632459
1764472,553144,97,8,NaN,152.621428,32.741494,0,2,469.000000,309.422126,0.000000,0.493898,1,NaN,513535,858.459370,218.076744,66.480565,11.401326,32.632459
1764473,553145,97,8,NaN,156.419841,31.602018,0,2,115.525440,422.474560,0.499352,0.499352,1,NaN,-535943,-569.342882,674.733425,-53.763184,42.350109,32.632459
1764474,553146,97,8,NaN,148.012119,27.402319,0,3,302.353245,451.335511,0.337981,0.333874,0,NaN,None,185.317256,791.312483,18.170614,51.554935,48.948688


### Calculate lifetimes

In [6]:
# Add life column
tabs_NoassLrg['Life'] = 0

# Calculate lifes
unique_id = np.unique(tabs_NoassLrg['id'])
for i in tqdm(np.arange(0, unique_id.shape[0])):
# for i in np.arange(0,5):
    snglFtr = tabs_NoassLrg[tabs_NoassLrg['id'] == unique_id[i]].copy().reset_index(drop=True)
    tabs_NoassLrg.loc[tabs_NoassLrg['id'] == unique_id[i],'Life'] = (np.max(snglFtr['frame'])-np.min(snglFtr['frame'])+1)



100%|██████████| 453705/453705 [2:05:02<00:00, 60.47it/s]  


### Drop single frame objects and change frame to time

In [7]:
tabs_NoassLrg.to_pickle('data\SWAMIS\\100_frames\swamis_NoAssociate_00045s.pnds')